In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
%matplotlib inline
# Our application logic will be added here

#if __name__ == "__main__":
#  tf.app.run()

/Users/Walkon302/anaconda2/envs/DS/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

### Data input

In [3]:
def preprocess(np_array):
    np_array = np.array(np_array/255., dtype = 'float32')
    np_array = np_array.reshape([len(np_array), np_array.shape[1]*np_array.shape[2]])
    return np_array

def data_generate(pos, neg, n):
    final = min(len(pos), len(neg))
    train_sample = np.concatenate([pos[0:n], neg[0:n]])
    eval_sample = np.concatenate([pos[n:final], neg[n:final]])
    train_label = np.concatenate([np.repeat(0, n), np.repeat(1, n)])
    eval_label = np.concatenate([np.repeat(0, final-n), np.repeat(1, final-n)])
    
    train_sample, train_label = shuffle(train_sample, train_label, random_state=0)
    eval_sample, eval_label = shuffle(eval_sample, eval_label, random_state=0)
    
    return train_sample, train_label, eval_sample, eval_label

In [4]:
good = np.load('test.npy')
bad = np.load('test_bad.npy')

a = preprocess(good)
b = preprocess(bad)
c = data_generate(a, b, 100)
train_sample, train_label, eval_sample, eval_label = data_generate(a, b, 150)

### Set up logging for prediction

In [5]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=10)

### Build the model

In [6]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
  # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 300, 300, 1])

  # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 75 * 75 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=2)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

### Build classifier

In [7]:
mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="/Users/Walkon302/Desktop/MEMA_organization/MEMA_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10ecdf950>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/Users/Walkon302/Desktop/MEMA_organization/MEMA_model', '_save_summary_steps': 100}


### Train the model

In [8]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_sample},
        y=train_label,
        batch_size=1,
        num_epochs=None,
        shuffle=True)

In [15]:
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=500,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /Users/Walkon302/Desktop/MEMA_organization/MEMA_model/model.ckpt-210
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 211 into /Users/Walkon302/Desktop/MEMA_organization/MEMA_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.23176113 0.7682389 ]]
INFO:tensorflow:loss = 0.26365453, step = 211
INFO:tensorflow:probabilities = [[0.41644284 0.5835572 ]] (12.554 sec)
INFO:tensorflow:probabilities = [[0.6685339  0.33146605]] (14.221 sec)
INFO:tensorflow:probabilities = [[0.75682396 0.243176  ]] (12.222 sec)
INFO:tensorflow:probabilities = [[0.8352845  0.16471554]] (11.619 sec)
INFO:tensorflow:probabilities = [[0.61202955 0.38797048]] (12.058 sec)
INFO:tensorflow:probabilities = [[0.53122824 0.4687718 ]] (12.684 sec)
INFO:ten

### Evaluate the model

In [10]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_sample},
    y=eval_label,
    num_epochs=1,
    shuffle=False)

In [16]:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-07-19:28:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /Users/Walkon302/Desktop/MEMA_organization/MEMA_model/model.ckpt-710
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-07-19:28:41
INFO:tensorflow:Saving dict for global step 710: accuracy = 0.68421054, global_step = 710, loss = 0.59245086


In [17]:
print(eval_results)

{'loss': 0.59245086, 'global_step': 710, 'accuracy': 0.68421054}
